<a href="https://colab.research.google.com/github/DarianSawali/Computer-Vision-Project-Overview/blob/main/Computer%20Vision%20YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q ultralytics opencv-python
import ultralytics
ultralytics.checks()


Ultralytics 8.3.218 🚀 Python-3.13.7 torch-2.9.0 CPU (Apple M4)
Setup complete ✅ (10 CPUs, 16.0 GB RAM, 79.7/460.4 GB disk)


In [9]:
import os

os.environ["ROBOFLOW_API_KEY"] = input("Enter your Roboflow API key (hidden): ")



Enter your Roboflow API key (hidden): 4NCLkwV8v1vJiR82S5q4


In [15]:
!pip3 -q install roboflow
from roboflow import Roboflow
import os

rf = Roboflow(api_key=os.environ["ROBOFLOW_API_KEY"])

project  = rf.workspace("sawali").project("taco-dataset-ql1ng-by2kn")
version  = project.version(1)
dataset  = version.download("yolov8")

data_yaml = dataset.location + "/data.yaml"
print("YAML:", data_yaml)


loading Roboflow workspace...
loading Roboflow project...
YAML: /Users/darianaxelsawali/TACO-dataset-1/data.yaml


In [18]:
from pathlib import Path
import yaml

data_yaml = Path.home() / "Downloads" / "TACO dataset.v1i.yolov8" / "data.yaml"
print("YAML exists:", data_yaml.exists(), "\nPath:", data_yaml)

with open(data_yaml, "r") as f:
    y = yaml.safe_load(f)

print("nc:", y.get("nc"))
print("names:", y.get("names"))

YAML exists: True 
Path: /Users/darianaxelsawali/Downloads/TACO dataset.v1i.yolov8/data.yaml
nc: 6
names: ['cardboard', 'glass', 'metal', 'other', 'paper', 'plastic']


In [19]:
from ultralytics import YOLO
import torch

device = "mps" if torch.backends.mps.is_available() else "cpu"
print("Using device:", device)

Using device: mps


In [23]:
model = YOLO("yolov8n.pt")

results = model.train(
    data=str(data_yaml),
    epochs=5,
    imgsz=512,
    batch=8,
    workers=0,
    patience=5,
    device=device
)

New https://pypi.org/project/ultralytics/8.3.221 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.218 🚀 Python-3.13.7 torch-2.9.0 MPS (Apple M4)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/darianaxelsawali/Downloads/TACO dataset.v1i.yolov8/data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=

In [24]:
metrics = model.val(device=device)
print(metrics.results_dict)

test_metrics = model.val(split="test", device=device)
print("Test mAP50:", test_metrics.results_dict.get("metrics/mAP50(B)"))
print("Figures saved in:", test_metrics.save_dir)

Ultralytics 8.3.218 🚀 Python-3.13.7 torch-2.9.0 MPS (Apple M4)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1486.5±747.9 MB/s, size: 1693.9 KB)
val: Scanning /Users/darianaxelsawali/Downloads/TACO dataset.v1i.yolov8/valid/labels.cache... 189 images, 4 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 189/189 844.2Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 12/12 1.5it/s 8.1s
                   all        189        346       0.51        0.2      0.153     0.0992
             cardboard         14         17      0.103      0.176       0.11     0.0736
                 glass         13         25          1          0       0.02     0.0111
                 metal         36         76      0.377      0.526      0.405      0.266
                 other          8          8          1          0     0.0115    0.00658
                 paper   

In [25]:
from glob import glob
demo_imgs = glob(str((data_yaml.parent / "test" / "images") / "*"))[:8]
preds = model.predict(source=demo_imgs, conf=0.25, save=True, device=device)
print("Annotated predictions saved in:", preds[0].save_dir)


0: 512x512 1 plastic, 170.0ms
1: 512x512 1 plastic, 170.0ms
2: 512x512 1 plastic, 170.0ms
3: 512x512 4 plastics, 170.0ms
4: 512x512 1 plastic, 170.0ms
5: 512x512 1 plastic, 170.0ms
6: 512x512 1 plastic, 170.0ms
7: 512x512 1 plastic, 170.0ms
Speed: 1.5ms preprocess, 170.0ms inference, 192.5ms postprocess per image at shape (1, 3, 512, 512)
Results saved to /Users/darianaxelsawali/runs/detect/predict
Annotated predictions saved in: /Users/darianaxelsawali/runs/detect/predict


In [28]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ultralytics.utils.metrics import ConfusionMatrix

metrics = model.val(device=device)

cm_path = list(metrics.save_dir.glob("confusion_matrix.npy"))
if cm_path:
    cm = np.load(cm_path[0])
    print("Loaded YOLO confusion matrix from:", cm_path[0])
else:
    cm = metrics.confusion_matrix.matrix
    print("Loaded from in-memory results")

with open(data_yaml, "r") as f:
    y = yaml.safe_load(f)
class_names = y["names"]

print("Classes:", class_names)

Ultralytics 8.3.218 🚀 Python-3.13.7 torch-2.9.0 MPS (Apple M4)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1159.9±591.4 MB/s, size: 1499.8 KB)
val: Scanning /Users/darianaxelsawali/Downloads/TACO dataset.v1i.yolov8/valid/labels.cache... 189 images, 4 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 189/189 523.2Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 12/12 1.5it/s 8.0s
                   all        189        346       0.51        0.2      0.153     0.0992
             cardboard         14         17      0.103      0.176       0.11     0.0736
                 glass         13         25          1          0       0.02     0.0111
                 metal         36         76      0.377      0.526      0.405      0.266
                 other          8          8          1          0     0.0115    0.00658
                 paper         38         48      0.297      0.097      0.111     0.0769
               p

In [29]:
plt.figure(figsize=(8,6))
sns.heatmap(
    cm,
    annot=True,
    fmt=".0f",
    cmap="Blues",
    xticklabels=class_names,
    yticklabels=class_names
)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("YOLOv8 Confusion Matrix — TACO Trash Sorter")
plt.show()


<Figure size 800x600 with 2 Axes>